In [5]:
import os

In [48]:
class Box:
    def __init__(self, name, conf, x1, y1, x2, y2):
        self.name = name
        self.conf = float(conf)
        self.x1 = float(x1)
        self.y1 = float(y1)
        self.x2 = float(x2)
        self.y2 = float(y2)

    def size2list(self):
        return [self.x1, self.y1, self.x2, self.y2]

def parse_txt(filepath, isGt):
    boxes = []
    method = None
    delay = None
    with open(filepath, "r") as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
        if not isGt:
            method, delay = lines[0].split()
            delay = float(delay)
            lines = lines[1:]

        for line in lines:
            split_line = line.split()
            boxes.append(Box(*split_line))
            
    return method, delay, boxes

In [35]:
def bbox_iou(box1, box2, x1y1x2y2=True):
    if x1y1x2y2:
        mx = min(box1[0], box2[0])
        Mx = max(box1[2], box2[2])
        my = min(box1[1], box2[1])
        My = max(box1[3], box2[3])
        w1 = box1[2] - box1[0]
        h1 = box1[3] - box1[1]
        w2 = box2[2] - box2[0]
        h2 = box2[3] - box2[1]
    else:
        w1 = box1[2]
        h1 = box1[3]
        w2 = box2[2]
        h2 = box2[3]

        mx = min(box1[0], box2[0])
        Mx = max(box1[0] + w1, box2[0] + w2)
        my = min(box1[1], box2[1])
        My = max(box1[1] + h1, box2[1] + h2)
    uw = Mx - mx
    uh = My - my
    cw = w1 + w2 - uw
    ch = h1 + h2 - uh
    carea = 0
    if cw <= 0 or ch <= 0:
        return 0.0

    area1 = w1 * h1
    area2 = w2 * h2
    carea = cw * ch
    uarea = area1 + area2 - carea
    return carea / uarea

In [64]:
def get_performance(result_dir, ground_true_dir="./gt"):
    track_sum = 0
    track_delay_sum = 0
    detect_sum = 0
    detect_delay_sum = 0
    track_scores_sum = 0
    detect_scores_sum = 0

    for _, _, files in os.walk(ground_true_dir):
        frame_count = len(files)
        for filename in files:
            gt_path = os.path.join(ground_true_dir, filename)
            result_path = os.path.join(result_dir, filename)
            method, delay, result_boxes = parse_txt(result_path, False)
            _, _, gt_boxes = parse_txt(gt_path, True)
            if method == "detect":
                detect_sum += 1
                detect_delay_sum += delay / 1e9
                detect_scores_sum += 100 
            else:
                track_sum += 1
                track_delay_sum += delay / 1e9
                temp_scores_sum = 0
                for result_box in result_boxes:
                    max_iou = 0
                    for gt_box in gt_boxes:
                        iou = bbox_iou(result_box.size2list(), gt_box.size2list())
                        if iou > max_iou:
                            max_iou = iou
                    temp_scores_sum += max_iou * 100
                    track_scores_sum += temp_scores_sum / (len(gt_boxes) * 100)
    detect_delay_avg = detect_delay_sum / detect_sum
    track_delay_avg = track_delay_sum / track_sum
    return detect_delay_avg, detect_delay_sum, track_delay_avg, track_delay_sum, detect_scores_sum, track_scores_sum

In [71]:
print(get_performance("./kcf"))
print(get_performance("./mosse"))
print(get_performance("./medainflow"))
print(get_performance("./csrt"))
print(get_performance("./mil"))
print(get_performance("./boosting"))
print(get_performance("./tld"))

(0.02654999999999999, 2.0443499999999992, 0.0460818051948052, 3.5482990000000005, 7700, 254.60564309760287)
(0.02703587012987013, 2.081762, 0.0066226753246753246, 0.509946, 7700, 60.30547752554178)
(0.026848194805194806, 2.067311, 0.01796651948051948, 1.383422, 7700, 321.4656107566765)
(0.02748387012987014, 2.1162580000000006, 0.27031870129870134, 20.81454, 7700, 341.45567704400503)
(0.027463259740259736, 2.1146709999999995, 0.378971012987013, 29.180768000000004, 7700, 327.6920769771779)
(1.2657376883116886, 97.46180200000002, 2.2438968441558433, 172.78005699999994, 7700, 213.3473582407876)
(0.02730925974025973, 2.1028129999999994, 0.44687837662337654, 34.409634999999994, 7700, 258.82112713349846)
